In [1]:
import numpy as np
import torch as th
import torchvision
import PIL
import cv2
import imageio
import blobfile as bf
import os
import tqdm

# Write Image

In [4]:
def _list_image_files_recursively(data_dir):
    results = []
    for entry in sorted(bf.listdir(data_dir)):
        full_path = bf.join(data_dir, entry)
        ext = entry.split(".")[-1]
        if "." in entry and ext.lower() in ["jpg", "jpeg", "png", "gif", "npy"] and not bf.isdir(full_path):
            results.append(full_path)
        elif bf.isdir(full_path):
            results.extend(_list_image_files_recursively(full_path))
    return results

def load_image(path):
    with bf.BlobFile(path, "rb") as f:
        pil_image = PIL.Image.open(f)
        pil_image.load()
    pil_image = pil_image.convert("RGB")
    return pil_image

def resize_arr(pil_image, image_size):
    img = pil_image.resize((image_size, image_size), PIL.Image.ANTIALIAS)
    return img

# Load Image

In [19]:
# src_dir = '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/'
# dst_dir = '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_128_tv_png2/valid/'

# src_dir = '/data/mint/DPM_Dataset/MultiPIE/MultiPIE_validset2/mp_aligned/valid/'
# dst_dir = '/data/mint/DPM_Dataset/MultiPIE/MultiPIE_validset2/mp_aligned_128/valid/'


# /data/mint/sampling/TPAMI/conditional_ablation/log=dpr_256_imgloss_cfg=dpr_256_imgloss/ema_200000/valid/render_face/src=001_02_01_051_14.png/dst=001_02_01_051_16.png/n_frames=2/res_frame0.png

src_dir = "/data/mint/sampling/TPAMI/conditional_ablation/log=dpr_256_imgloss_cfg=dpr_256_imgloss/ema_500000/"
dst_dir = "/data/mint/sampling/TPAMI/conditional_ablation/log=dpr_128_imgloss_cfg=dpr_128_imgloss/ema_500000/"
os.makedirs(dst_dir, exist_ok=True)

src_img_path = _list_image_files_recursively(src_dir)
for p in tqdm.tqdm(src_img_path):
    # Get the path after ema_200000
    p_dst = '/'.join(p.split('ema_500000/')[-1].split('/')[:-1])
    img_name = p.split('/')[-1].split('.')[0]
    pil_img = load_image(p)
    pil_img = pil_img.resize((128, 128), PIL.Image.ANTIALIAS)
    np_img = np.array(pil_img).transpose((2, 0, 1))
    os.makedirs(f'{dst_dir}/{p_dst}', exist_ok=True)
    torchvision.utils.save_image(th.tensor(np_img/255.0), f'{dst_dir}/{p_dst}/{img_name}.png')
    # pil_img.save(f'{dst_dir}/{img_name}.jpg')

100%|██████████| 600/600 [00:07<00:00, 83.46it/s] 
